In [1]:
!pip install langchain==0.2.0 --quiet
!pip install langchain-openai==0.1.7 --quiet
!pip install langchain-community==0.2.0 --quiet
!pip install sentence-transformers==2.7.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
!pip install langchain-chroma

In [1]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

In [2]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [3]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

In [4]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW download it
# manually upload it on colab
!gdown 1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW

Downloading...
From (original): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW
From (redirected): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW&confirm=t&uuid=9b1308b2-7bc3-4667-a6f2-f56daa7e4598
To: /content/simplewiki-2020-11-01.jsonl.gz
100% 50.2M/50.2M [00:01<00:00, 37.1MB/s]


In [5]:
import gzip
import json

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

docs = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())

        #Add all paragraphs
        #passages.extend(data['paragraphs'])

        #Only add the first paragraph
        docs.append({
                        'metadata': {
                                        'title': data.get('title'),
                                        'article_id': data.get('id')
                        },
                        'data': ' '.join(data.get('paragraphs')[0:3]) # restrict data to first 3 paragraphs to run later modules faster
        })

print("Passages:", len(docs))

Passages: 169597


In [6]:
# We subset our data so we only use a subset of wikipedia documents to run things faster
docs = [doc for doc in docs for x in ['linguistics', 'india', 'cheetah']
              if x in doc['data'].lower().split()]

In [7]:
len(docs)

1364

In [8]:
docs[:3]

[{'metadata': {'title': 'Kurgan hypothesis', 'article_id': '72554'},
  'data': 'The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'},
 {'metadata': {'title': 'Marija Gimbutas', 'article_id': '72558'},
  'data': 'Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'},
 {'metadata': {'title': 'Basil', 'article_id': '73985'},
  'data': 'Basil ("Ocimum basilic

### Create LangChain Documents

In [9]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc['data'],
                 metadata=doc['metadata']) for doc in docs]

In [10]:
docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'article_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'article_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content

In [11]:
len(docs)

1364

### Split larger documents into smaller chunks

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

In [13]:
chunked_docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'article_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language. It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion. The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'article_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content

In [14]:
len(chunked_docs)

1388

In [15]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='rag_wikipedia_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wikipedia_db")

In [16]:
# load from disk
chroma_db = Chroma(persist_directory="./wikipedia_db",
                   collection_name='rag_wikipedia_db',
                   embedding_function=openai_embed_model)

In [17]:
chroma_db

In [18]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [19]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever

# Retriever 1 - simple cosine distance based retriever
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)
# Retriever 2 - retrieves the documents similar to query and then applies the filter
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=similarity_retriever
)

# download an open-source reranker model - BAAI/bge-reranker-v2-m3
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-large")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=3)
# Retriever 3 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker_compressor, base_retriever=compressor_retriever
)

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [21]:
final_retriever

ContextualCompressionRetriever(base_compressor=CrossEncoderReranker(model=HuggingFaceCrossEncoder(client=<sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder object at 0x782a8429bf90>, model_name='BAAI/bge-reranker-large', model_kwargs={}), top_n=3), base_retriever=ContextualCompressionRetriever(base_compressor=LLMChainFilter(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=BooleanOutputParser(), template="Given the following question and context, return YES if the context is relevant to the question and NO if it isn't.\n\n> Question: {question}\n> Context:\n>>>\n{context}\n>>>\n> Relevant (YES / NO):"), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x782a7fc3bc90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x782a7fc4b8d0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')), get_input=<function default_get_input at 0x782a7cabc360>), base_ret

In [20]:
query = "what is the old capital of India?"
docs = final_retriever.invoke(query)
docs

[Document(id='4d94cf26-6d2c-471c-b6d2-88ae3e8343c0', metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once the center of industry and education. However, it has witnessed political violence and economic problems since 1954. Since 2000, Kolkata has grown due to economic growth. Like other metropolitan cities in India, Kolkata struggles with poverty, pollution and traffic congestion. The discovery of the nearby Chandraketugarh, an archaeological site has proved that people have lived there for o

In [22]:
query = "what is the fastest animal?"
docs = final_retriever.invoke(query)
docs

[Document(id='449ee7ee-c551-4a94-b26e-f2ea42c25455', metadata={'article_id': '9800', 'title': 'Cheetah'}, page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening. The cheetah compared to other big cats is light and slimly built. Its long thin legs and long spotted tail are necessary for fast running. Its lightly built, thin form is in sharp contrast with the robust build of other big cats. The head-and-body length ranges from . The cheetah stands 70 to 90\xa0cm at the shoulder, and weighs . The slightly curved claws are only weakly retractable (semi-retractable). This is a major point of difference between the cheetah and the other big cats, which have fully retractable claws.'),
 Document(id='adf04103-65be-4eea-9

## Build a QA RAG Chain

To build a RAG chain we need a prompt template which instructs the LLM to not answer questions beyond the scope of the retrieved context documents, there are various such prompts out there, we craft one ourselves below

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know.
            Do not make up the answer unless it is there in the provided context.
            Give a detailed answer with regard to the question.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

## LCEL Syntax for QA RAG Chain - Recommended

Here we show you how to create the RAG chain using LangChain's recommended LCEL

In [24]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (final_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    prompt_template
      |
    chatgpt
)

In [25]:
from IPython.display import Markdown, display

query = "What is the financial capital of India?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The financial capital of India is Mumbai. It is the largest city in India and one of the world's most populous cities. Mumbai generates more than 6% of India's GDP and accounts for 25% of industrial output, 40% of sea trade, and 70% of capital to India's economy. The city is home to the Reserve Bank of India, the Bombay Stock Exchange, the National Stock Exchange of India, and many Indian companies and multinational corporations. Additionally, the headquarters of the State Bank of India, the largest bank in India, are located in Mumbai. Therefore, Mumbai is widely recognized as the financial capital of India.

In [26]:
query = "What is the old capital of India?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The old capital of India was Kolkata (formerly known as Calcutta) during the British Raj until 1911. Kolkata served as the capital of British India from 1772 until the capital shifted to the hilly town of Shimla during the summer months every year starting from 1864. Kolkata was a significant center of industry, education, and trade during the British rule. It was declared a Presidency City and later became the headquarters of the Bengal Presidency. The city played a crucial role in the British East India Company's opium trade during the 18th and 19th centuries.

In [27]:
query = "Tell me about the fastest animal?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The fastest animal is the cheetah, specifically the South African Cheetah ("Acinonyx jubatus jubatus"). It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Cheetahs are medium-large cats that live in Africa, primarily in the savannas. They are active during the day and hunt in the early morning or late evening. The cheetah's long thin legs, long spotted tail, and light, slim build are necessary for its incredible speed. Compared to other big cats, the cheetah is lightly built and has slightly curved, weakly retractable claws, which is a major difference from other big cats with fully retractable claws. The South African Cheetah is the nominate subspecies of cheetah native to Southern Africa and is the most abundant subspecies, with an estimated population of more than 6,000 individuals in the wild.

In [28]:
query = "Explain linguistics in simple terms"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Linguistics is the study of language. Linguists study various aspects of language, including sounds (phonology), parts of words (morphology), word order and sentence structure (syntax), word meanings (semantics), and the hidden meanings behind speech (pragmatics). There are two main branches of linguistics: theoretical linguistics, which focuses on understanding the theory and ideas behind language, and applied linguistics, which uses linguistic knowledge for practical purposes. Theoretical linguistics explores how language works and the common features found in all languages. Applied linguistics includes fields like forensic linguistics, used in crime investigations, and computational linguistics, which involves making computers understand human language through sub-fields like speech recognition and speech synthesis.

In [29]:
query = "Who won the champions league in 2021"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.